# Municípios do Brasil

O IBGE disponibiliza diversos dados abertos, muitos deles através de APIs. Tirando proveito disso é possível pegar todos os municípios do Brasil e seus respectivos estados para criar um dicionário.

In [1]:
import pandas as pd

In [2]:
url = 'https://servicodados.ibge.gov.br/api/v1/localidades/municipios?orderBy=nome'

In [3]:
municipios = pd.read_json(url)

Como existe muitas cidades com o mesmo nome, vou criar uma lista de prioridade seguindo a regra: `capital > outros municípios` e `municípios de goiás > demais`, a motivação por trás disso é simples, é mais provável que uma pessoa seja de um município que é capital do que de outros municípios, pois são mais populosos e a probabilidade dessas pessoas serem do estado de Goiás é maior pois é onde o programa funciona (as compras precisam ser feitas no estado para concorrerem).

In [4]:
capitais = {
    'Acre': 'Rio Branco',
    'Alagoas': 'Maceió',
    'Amapá': 'Macapá',
    'Amazonas': 'Manaus',
    'Bahia': 'Salvador',
    'Ceará': 'Fortaleza', 
    'Distrito Federal': 'Brasília',
    'Espírito Santo': 'Vitória',
    'Goiás': 'Goiânia',
    'Maranhão': 'São Luís',
    'Mato Grosso': 'Cuiabá',
    'Mato Grosso do Sul': 'Campo Grande',
    'Minas Gerais': 'Belo Horizonte',
    'Pará': 'Belém',
    'Paraíba': 'João Pessoa',
    'Paraná': 'Curitiba',
    'Pernambuco': 'Recife', 
    'Piauí': 'Teresina',
    'Rio de Janeiro': 'Rio de Janeiro',
    'Rio Grande do Norte': 'Natal',
    'Rio Grande do Sul': 'Porto Alegre',
    'Rondônia': 'Porto Velho',
    'Roraima': 'Boa Vista',
    'Santa Catarina': 'Florianópolis',
    'São Paulo': 'São Paulo',
    'Sergipe': 'Aracaju',
    'Tocantins': 'Palmas' 
}

In [5]:
municipios['uf'] = pd.json_normalize(municipios.microrregiao)['mesorregiao.UF.nome']

In [6]:
municipios.drop(columns=['microrregiao', 'regiao-imediata'], inplace=True)
municipios.head()

,id,nome,uf
0,5200050,Abadia de Goiás,Goiás
1,3100104,Abadia dos Dourados,Minas Gerais
2,5200100,Abadiânia,Goiás
3,3100203,Abaeté,Minas Gerais
4,1500107,Abaetetuba,Pará


In [7]:
municipios['capital'] = 0

In [8]:
for key in capitais.keys():
    municipios.loc[municipios[(municipios.uf == key) & (municipios.nome == capitais[key])].index, 'capital'] = 1

In [9]:
municipios['goias'] = [int(x == 'Goiás') for x in municipios.uf]

In [10]:
municipios.sort_values(by=['capital', 'goias'], ascending=False, inplace=True)

In [11]:
municipios.drop_duplicates(subset=['nome'], keep='first', inplace=True)

In [12]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [13]:
municipios['nome_simples'] = [strip_accents(i) for i in municipios.nome.str.upper()]
municipios['uf_simples'] = [strip_accents(i) for i in municipios.uf.str.upper()]

In [14]:
municipios.drop(columns=['capital','goias'], inplace=True)

In [15]:
municipios.head()

,id,nome,uf,nome_simples,uf_simples
1926,5208707,Goiânia,Goiás,GOIANIA,GOIAS
294,2800308,Aracaju,Sergipe,ARACAJU,SERGIPE
579,1501402,Belém,Pará,BELEM,PARA
591,3106200,Belo Horizonte,Minas Gerais,BELO HORIZONTE,MINAS GERAIS
642,1400100,Boa Vista,Roraima,BOA VISTA,RORAIMA


In [16]:
municipios.to_csv('dados/municipios.csv', index=False)